In [ ]:
import pathlib
import pandas as pd

datadir = pathlib.Path.cwd().parent / "data"

pd_aozora = pd.read_parquet(datadir / "aozora.parquet")

# 13章 文字列
## 13-1 部分文字列の検出・抽出・置換
### Q: URLから芥川竜之介の作品を判定して抽出


#### Awesome

In [ ]:
pd_aozora.loc[lambda df: df.url.str.contains("/000879/")]

### Q: 出版年月から年と月を位置指定で抽出


#### Awesome

In [ ]:
(
    pd_aozora
    .assign(
        # published_atの文字列の一部を抽出
        year=lambda df: df.published_at.str[:4],
        month=lambda df: df.published_at.str[5:7]
    )
    [["text_id", "title", "published_at", "year", "month"]]
)

### Q: 出版年月のフォーマットを`YYYY年MM月`から`YYYY-MM`に変換（部分文字列の検出、置換）


#### Awesome

In [ ]:
import numpy as np

(
    pd_aozora
    .assign(published_at=lambda df:
        # published_atが"月"を含む場合は"年"を"-"に置換して"月"を削除、それ以外は"年"を削除
        np.where(df.published_at.str.contains("月"),
            df.published_at.str.replace("年", "-").str.replace("月", ""),
            df.published_at.str.replace("年", ""))
    )
    [["text_id", "title", "published_at"]]
)

### Q: URLから人物IDと作品IDを抽出 (正規表現による抽出)


#### Awesome

In [ ]:
(
    pd_aozora
    .assign(
        # urlから正規表現で抽出
        author_id=lambda df: df.url.str.extract(r"cards/0*([1-9][0-9]*)/"),
        text_id=lambda df: df.url.str.extract(r"card([0-9]+)\.")
    )
    [["title", "url", "author_id", "text_id"]]
)

## 13-2 区切り文字による文字列分割
### Q: 句点で文に分割


#### Awesome

In [ ]:
(
    pd_aozora
    # "。"でtextを分割
    .assign(sentence_list=lambda df: df.text.str.split("。"))
    [["text_id", "title", "sentence_list"]]
)

### Q: 複数の区切り文字で文に分割


#### Awesome

In [ ]:
(
    pd_aozora
    # "。"または"\n"でtextを分割
    .assign(sentence_list=lambda df: df.text.str.split(r"。|\n"))
    [["text_id", "title", "sentence_list"]]
)

### Q: 分割結果の配列を列に展開


#### Awesome

In [ ]:
(
    pd_aozora
    .assign(
        #（1）"。"でtextを分割
        sentence_list=lambda df: df.text.str.split("。"),
        #（2）sentence_listの0番目と1番目の要素を取得
        first_sentence=lambda df: df.sentence_list.str[0],
        second_sentence=lambda df: df.sentence_list.str[1]
    )
    [["text_id", "title", "first_sentence", "second_sentence"]]
)

### Q: 分割結果の配列を行に展開


#### Awesome

In [ ]:
(
    pd_aozora
    #（1）"。"でtextを分割
    .assign(sentence=lambda df: df.text.str.split("。"))
    [["text_id", "title", "sentence"]]
    #（2）sentenceを複数行に展開
    .explode("sentence")
)

## 13-3 文字列データのクレンジング
### Q: ユニコード正規化


#### Awesome

In [ ]:
(
    pd_aozora
    # textをユニコード正規化
    .assign(text=lambda df: df.text.str.normalize("NFKC"))
    [["text_id", "title", "text"]]
)

### Q: 不要な文字列パターンの除去（正規表現による除去）


#### Awesome

In [ ]:
pd_aozora_after_cleansing = (
    pd_aozora
    # （1） 文への分割と行展開
    .assign(sentence=lambda df: df.text.str.split("。|\n"))
    [["text_id", "title", "sentence"]]
    .explode("sentence") 
    .assign(
        sentence=lambda df:
            df
            # （2） ユニコード正規化
            .sentence.str.normalize("NFKC")
            # （3） 正規表現による不要パターンの除去
            .str.replace(r"《.*?》|\[#.*?\]|[\|※「」()]", "", regex=True)
            # （4） 先頭や末尾のスペースの除去
            .str.strip()
    )
    # （5） 長さが1文字以上の行のみ抽出
    .query("sentence.str.len() >= 1")
)
pd_aozora_after_cleansing


## 13-4 形態素解析による日本語文章の単語分解と単語抽出
### Q: 文を単語に分解（形態素解析）
#### Awesome

In [ ]:
# spaCyおよびGiNZAモジュールの読み込み
# 初回はpipでインストールする
# pip install spacy ja-ginza
import spacy
nlp = spacy.load("ja_ginza")

pd_aozora_token = (
    pd_aozora_after_cleansing
    # （1） 単語（Tokenオブジェクト）への分解と行展開
    .assign(token=lambda df: list(nlp.pipe(df.sentence)))
    [["text_id", "token"]]
    .explode("token")
    # （2） Tokenオブジェクトのプロパティから列を作成
    .assign(
        # 元の文章での表記
        text=lambda df: df.token.apply(lambda e: e.text),
        # 単語の原型
        lemma=lambda df: df.token.apply(lambda e: e.lemma_),
        # 単語の品詞
        pos=lambda df: df.token.apply(lambda e: e.pos_),
        # 単語がストップワードかどうかを表すブール値
        is_stop=lambda df: df.token.apply(lambda e: e.is_stop),
        # 単語にベクトル表現が付与されているかどうかを表すブール値
        has_vector=lambda df: df.token.apply(lambda e: e.has_vector),
        # 単語のベクトル表現 （付与されていない場合はゼロベクトル)
        vector=lambda df: df.token.apply(lambda e: e.vector)
    )
    # （3） 不要な列の削除、Index修正
    .drop(columns="token")
    .reset_index(drop=True)
)
pd_aozora_token

### Q: 特定の品詞の抽出とストップワードの除去
#### Awesome

In [ ]:
# 名詞と固有名詞の抽出とストップワードの除去
pd_aozora_token.query("pos in ('NOUN', 'PROPN') and not is_stop")

## 13-5 文章のベクトル化
### Q: bag-of-wordsベクトルの作成
#### Awesome

In [ ]:
(
    pd_aozora_token
    # （1） 名詞と固有名詞の抽出とストップワードの除去
    .query("pos in ('NOUN', 'PROPN') and not is_stop")
    # （2） text_idごとにlemmaの値をカウントし、横持ち行列に変換
    .groupby(["text_id", "lemma"]).size()
    .unstack()
)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# text_idごとの単語リストを作成
pd_word_list = (
    pd_aozora_token
    .query("pos in ('NOUN', 'PROPN') and not is_stop")
    # text_idごとにlemmaのlistを作成
    .groupby("text_id").lemma.apply(list)
)

# 単語リストを入力する場合は、analyzerにlambda x: xを渡す必要がある
vectorizer = CountVectorizer(analyzer=lambda x: x)
# 単語リストを渡して出現回数をカウント
vector = vectorizer.fit_transform(pd_word_list).toarray()
# 出力用に整形
pd.DataFrame(vector, columns=vectorizer.get_feature_names_out(), index=pd_word_list.index)

### Q: TF-IDF 特徴量の作成
#### Awesome

In [ ]:
import numpy as np

(
    pd_aozora_token
    # （1） 品詞の抽出とストップワードの除去
    .query("pos in ('NOUN', 'PROPN') and not is_stop")
    # （2） text_idごとにlemmaの値をカウント
    .groupby(["text_id", "lemma"]).size().rename("word_count")
    # （3） TF-IDFを計算
    .to_frame()
    .reset_index()
    .assign(
        # （3）-1 文章ごとに全単語の出現回数の合計
        total_word_count=lambda df:
            df.groupby("text_id").word_count.transform("sum"),
        # （3）-2 単語ごとにその単語が含まれる文章数
        text_count_with_word=lambda df:
            df.groupby("lemma").text_id.transform("nunique"),
        # （3）-3 全文章数
        text_count=lambda df: df.text_id.nunique(),
        # （3）-4 TF=ある文章における対象単語の出現回数/ある文章に含まれる全単語の出現回数の合計
        tf=lambda df: df.word_count / df.total_word_count,
        # （3）-5 IDF=log(全文章数/対象単語が含まれる文章数) + 1
        idf=lambda df: np.log(df.text_count / df.text_count_with_word) + 1,
        # （3）-6 TF-IDF=TF*IDF
        tf_idf=lambda df: df.tf * df.idf,
        # （3）-7 TF-IDFを文章ごとに計算したL2ノルムで割って正規化
        tf_idf_normalized=lambda df:
            df.tf_idf / df.groupby("text_id").tf_idf.transform(np.linalg.norm)
    )
    # （4） TF-IDFの列のみに絞り、横持ち行列に変換
    .set_index(["text_id", "lemma"])["tf_idf_normalized"]
    .unstack()
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# text_idごとの単語リストを作成
pd_word_list = (
    pd_aozora_token
    .query("pos in ('NOUN', 'PROPN') and not is_stop")
    # text_idごとにlemmaのlistを作成
    .groupby("text_id").lemma.apply(list)
)

# 単語リストを入力する場合は、analyzerにlambda x: xを渡す必要がある
# デフォルトではゼロ除算を防ぐためにsmooth_idf=Trueになるが、上のコードと若干数値がずれるためFalseに設定
vectorizer = TfidfVectorizer(analyzer=lambda x: x, smooth_idf=False)
# 単語リストを渡して出現回数をカウント
vector = vectorizer.fit_transform(pd_word_list).toarray()
# 出力用に整形
pd.DataFrame(vector, columns=vectorizer.get_feature_names_out(),
             index=pd_word_list.index)

### Q: 平均単語ベクトル特徴量の作成
#### Awesome

In [ ]:
(
    pd_aozora_token
    # （1） 品詞の抽出とストップワードの除去
    .query("pos in ('NOUN', 'PROPN') and not is_stop")
    # （2） 作品ごとに単語ベクトルの平均を計算
    .groupby("text_id").vector.mean()
)